# Bokeh

This open source project allows highly interact graphs to be created from python and displayed in the browser using a custom JavaScript back end. Jupyter notebooks are well supported but other options are available.

In [1]:
import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import factor_cmap, factor_mark
bokeh.__version__

'1.0.4'

Start with some data in a pandas dataframe. Bokeh will happily use this, but its underlying structure is based on dictionaries.

In [80]:
import numpy as np
import pandas as pd
data = pd.DataFrame(np.load('sdss_colors.npy'))
data['u-g'] = data['u'] - data['g']
data['g-r'] = data['g'] - data['r']
data['r-i'] = data['r'] - data['i']

galaxies = data[data['specClass'] == 'GALAXY']
qsos = data[data['specClass'] == 'QSO']

data[:3]

,u,g,r,i,z,specClass,redshift,redshift_err,u-g,g-r,r-i
0,18.06726,17.92133,17.76799,17.83525,17.74435,QSO,0.946076,0.000269,0.14593,0.15334,-0.06726
1,19.46527,17.84728,17.00071,16.56581,16.20629,GALAXY,0.078311,0.000021,1.61799,0.84657,0.43490
2,18.65488,17.34237,16.79778,16.49659,16.24572,GALAXY,0.032940,0.000019,1.31251,0.54459,0.30119


Create a basic scatter plot. Interactivity is fairly limited: zoom, pan and .PNG save can be enabled with the toolbar (click to toggle, hover to identify the button), and `legend.click_policy` lets you selectively hide/unhide series.

In [6]:
# output to notebook cell
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="Galaxies and QSOs", x_axis_label='u-g', y_axis_label='g-r')
p.scatter('u-g', 'g-r', source=galaxies, legend='specClass', color='red')
p.scatter('u-g', 'g-r', source=qsos, legend='specClass')
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

Tooltips are easy to add, providing extra information when you hover the mouse. This example is pretty basic but there are more configuration options.

Rows in the tooltip are (label, value) tuples. Some standard values start with `$`, dataframe fields with `@`.

In [81]:
TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("redshift", "@redshift +- @redshift_err"),
]

p2 = figure(title="Galaxies and QSOs", 
           x_axis_label='u-g', y_axis_label='g-r', 
           tooltips=TOOLTIPS)
p2.scatter('u-g', 'g-r', source=galaxies, legend='specClass', color='red')
p2.scatter('u-g', 'g-r', source=qsos, legend='specClass')
p2.legend.click_policy="hide"
show(p2)

## Linking Plots

Multiple graphs can be laid out in a gridplot. If appropriate, the axis ranges can be linked so that plots pan and zoom in sync.

In [82]:
from bokeh.layouts import gridplot

left = figure(title="Galaxies and QSOs 1", 
           x_axis_label='u-g', y_axis_label='g-r', 
           plot_width=400, plot_height=400, 
           tooltips=TOOLTIPS)
left.scatter('u-g', 'g-r', source=galaxies, legend='specClass', color='red')
left.scatter('u-g', 'g-r', source=qsos, legend='specClass')
left.legend.click_policy="hide"

right = figure(title="Galaxies and QSOs 2", 
           x_axis_label='g-r', y_axis_label='r-i', 
           plot_width=400, plot_height=400,
           x_range=left.x_range, y_range=left.y_range,
           tooltips=TOOLTIPS)
right.scatter('g-r', 'r-i', source=galaxies, legend='specClass', color='red')
right.scatter('g-r', 'r-i', source=qsos, legend='specClass')
right.legend.click_policy="hide"

p3= gridplot([[left, right]])

show(p3)

## Custom JavaScript

If pure Python isn't doing everything you need, Bokeh makes it relatively easy to add snippets of JS script as callbacks. Assuming you can deal with a bit of JS programming (it makes you appreciate Python more).

The next demo is taken from one of their examples. It uses a full-image plot to keep track of where a linked plot is zoomed to: useful orientation when play with complex data.

In [24]:
from bokeh.models import ColumnDataSource, CustomJS, Rect
from bokeh.layouts import row

# Details of the rectangle to draw dynamically
source = ColumnDataSource({'x':[], 'y':[], 'width':[], 'height':[]})

# Custom JS for the callback
JSCODE = """
var data = source.data;
var start = cb_obj.start;
var end = cb_obj.end;
data[%r] = [start + (end - start) / 2];
data[%r] = [end - start];

// this is needed because we modified .data "in place"
source.change.emit();
"""

In [28]:
left = figure(title="Zoom and Pan Here", 
           x_axis_label='u-g', y_axis_label='g-r', 
           x_range=(-1, 2.5), y_range=(-0.6, 1.7),
           plot_width=400, plot_height=400, 
           tooltips=TOOLTIPS)
left.scatter('u-g', 'g-r', source=galaxies, legend='specClass', color='red')
left.scatter('u-g', 'g-r', source=qsos, legend='specClass')
left.legend.click_policy="hide"

left.x_range.callback = CustomJS(args=dict(source=source), code=JSCODE % ('x', 'width'))
left.y_range.callback = CustomJS(args=dict(source=source), code=JSCODE % ('y', 'height'))

right = figure(title="See Zoom Window Here", 
           x_axis_label='u-g', y_axis_label='g-r',
           x_range=(-1, 2.5), y_range=(-0.6, 1.7),
           plot_width=400, plot_height=400, tools="")
right.scatter('u-g', 'g-r', source=galaxies, color='red')
right.scatter('u-g', 'g-r', source=qsos)

# Add a rectangle corresponding to the left plot
right.rect('x', 'y', 'width', 'height', fill_alpha=0, line_color='black', source=source)

show(row(left, right))

## Interactive Widgets

It is of course possible to use IPython Interact widgets to redraw the plot from scratch, exactly as with Matplotlib. Bokeh, with its roots in JavaScript, provides more options for client-side interactivity. This works best when using a Bokeh server, but a subset of options are compatible with Jupyter.



Still using ipywidgets, it is possible to change the data being plotted then call `push_notebook()` to refresh the plot without redrawing axes, labels, etc.

In [30]:
from ipywidgets import interact
import ipywidgets as w
from bokeh.io import push_notebook

In [112]:
p4 = figure(title="Galaxies and QSOs", 
           x_axis_label='u-g', y_axis_label='g-r')
g = p4.scatter(x=galaxies['u-g'], y=galaxies['g-r'], legend='Galaxies', color='red')
q = p4.scatter(x=qsos['u-g'], y=qsos['g-r'], legend='QSOs')

All columns of the graph's source data ___must___ be updated simultaneously, most conveniently by passing a dictionary. Changing x then y generates lots of error messages about mismatched lengths.

In [113]:
def update(z):
    gal = galaxies[galaxies['redshift'].between(z[0], z[1])] 
    qso = qsos[qsos['redshift'].between(z[0], z[1])] 

    new_gal = {'x' : gal['u-g'], 'y' : gal['g-r']}
    new_qso = {'x' : qso['u-g'], 'y' : qso['g-r']}

    g.data_source.data = new_gal
    q.data_source.data = new_qso
    push_notebook();

In [118]:
show(p4, notebook_handle=True)

interact(update, 
         z = w.FloatRangeSlider(description="redshift", 
                    value=[0, 6], step=0.05, min=0, max=6));

interactive(children=(FloatRangeSlider(value=(0.0, 6.0), description='redshift', max=6.0, step=0.05), Output()…

That isn't super-exciting so far, but it opens the possibility of continuously updating the data to create animations. Matplotlib can't do that in real time.

There's no animation demo in this notebook, but a separate set on animations is planned.

### Built-in widgets

Bokeh has a variety of widgets built in, based on the Bootstrap framework and entirely distinct from ipywidgets. However, these are designed for use with a Bokeh server or custom JavaScript with `on_change()` callbacks, and are not easily usable with Jupyter. So the next cell is entirely useless!

In [105]:
from bokeh.layouts import widgetbox
from bokeh.models.widgets import RangeSlider

range_slider = RangeSlider(start=0, end=10, value=(1,9), step=.1, title="Stuff")

show(widgetbox(range_slider))